In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt 

### Hypothesese: 
1) sales and oil prices have inverse relationship <br>
2) local holidays will affect prices for local grocery stores <br>
3) onpromotion and unit sales are positively correlated <br>
### Questions
1) is there a correlation between onpromotion and perishable? <br>
2) create classification for onpromotion -- use true and false values to predict null values 

In [2]:
# Import Data
train = pd.read_csv('train-1.csv')
holiday_events = pd.read_csv('holidays_events.csv')
items = pd.read_csv('items.csv')
oil = pd.read_csv('oil.csv')
stores = pd.read_csv('stores.csv')
transactions = pd.read_csv('transactions.csv')

test = pd.read_csv('test-1.csv')
sample_submission = pd.read_csv('sample_submission-1.csv')

/Users/muronghe/anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
train.subset = train.subset('')

,id,date,store_nbr,item_nbr,unit_sales,onpromotion
0,0,2013-01-01,25,103665,7.0,NaN
1,1,2013-01-01,25,105574,1.0,NaN
2,2,2013-01-01,25,105575,2.0,NaN
3,3,2013-01-01,25,108079,1.0,NaN
4,4,2013-01-01,25,108701,1.0,NaN


### Create subset of Training Data

In [4]:
#create copy of training data 
train_original = train

In [6]:
#convert date to datetime
train.loc[:,'date']  = pd.to_datetime(train.loc[:,'date'])

In [8]:
#Create subset of training data (for 2016)
train = train[train.date.dt.year==2016]

In [9]:
#total data in training set 
train.shape

(35229871, 6)

## Data Exploration 

## 6 tables:
#### 1) train 
id<br>
date<br>
store_nbr<br>
item_nbr <br>
unit_sales <br>
onpromotion <br>

#### 2) oil 
date <br>
dcoilwticol <br>

#### 3) holidays_events
date<br>
type<br>
locale<br>
locale_name<br>
description<br>
transferred<br>

#### 4) items
item_nbr<br>
family<br>
class<br>
perishable<br>

#### 5) transactions
date<br>
store_nbr<br>
transactions<br>

#### 6) stores
store_nbr<br>
city<br>
state<br>
type<br>
cluster<br>

In [7]:
# onpromotion count in train dataset -- there are many null values
train['onpromotion'].value_counts(dropna = False)

False    96028767
NaN      21657651
True      7810622
Name: onpromotion, dtype: int64

In [10]:
# where do the null values in the training dataset fall? 
train[train['onpromotion'].isnull()].groupby(train.date.dt.year, as_index = False).count()

,id,date,store_nbr,item_nbr,unit_sales,onpromotion


In [ ]:
# onpromotion count in test dataset -- there are no null values
test['onpromotion'].value_counts(dropna=False)

In [ ]:
#groupby training data 
train_groupbydate = train.groupby('date', as_index=False).sum()
train_groupbydate = train_groupbydate.sort_values('date', ascending = True)

In [ ]:
#plot total unit sales by date
fig = plt.plot(train_groupbydate['date'],train_groupbydate['unit_sales'])
plt.show()

In [ ]:
#holiday types
holiday_events['type'].value_counts() 

In [ ]:
#number unique states -- there are 24 states in ecuador -- not all are in this dataset
len(stores['state'].unique())

In [ ]:
#count of states
stores.groupby(['state'])['state'].count()

In [ ]:
stores['state'].value_counts()

### Dates 
Training dataset is from 1/1/2013 - 8/15/2017 <br>
Test dataset is from 8/16/2017 - 8/31/2017 <br>
Oil data is from 1/1/2013 - 8/31/2017 <br>
Transaction data is from 1/1/2013 - 8/15/2017 <br>
Holiday events is from 3/2/2012 - 12/26/2017

In [ ]:
print(train.date.min())
print(train.date.max())

In [ ]:
print(test.date.min())
print(test.date.max())

In [ ]:
print(oil.date.min())
print(oil.date.max())

In [ ]:
print(transactions.date.min())
print(transactions.date.max())

In [ ]:
print(holiday_events.date.min())
print(holiday_events.date.max())

In [ ]:
# Store Types
stores.groupby(['type'])['type'].count()

In [ ]:
items.groupby(['family'])['family'].count()

In [ ]:
# Plot of items.family
k = items.groupby(['family'])['family'].count()
k.plot.bar(
    subplots = True, x = items['family'], figsize = (10,5), legend = None)
plt.show()

### Row Counts

In [ ]:
print(len(transactions))
print(len(stores))
print(len(holiday_events))
print(len(oil))
print(len(items))
print(len(train))

# Convert Dates to DateTime

In [6]:
#Change Date to datetime
def convert_date(table):
    table['date'] = pd.to_datetime(table['date'])

In [ ]:
convert_date(transactions)
convert_date(oil)
convert_date(holiday_events)
convert_date(train)
convert_date(test)

# Merge Tables

In [15]:
#merge tables 
def merge_all_columns(train_test,transactions,oil,holiday_events,items,stores):
    #merge train/test with transactions
    merged_df = train_test.merge(transactions, how = 'left', left_on = [train_test['date'],
                                                               train_test['store_nbr']], 
                        right_on = [transactions['date'],transactions['store_nbr']])
    #merge with oil data
    merged_df = merged_df.merge(oil, how='left', left_on = merged_df['date_x'], 
                            right_on = oil['date'])
    #merge with holiday_events
    merged_df = merged_df.merge(holiday_events, how = 'left', left_on = merged_df['date_x'],
                           right_on = holiday_events['date'])
    #merge with items
    merged_df = merged_df.merge(items, how = 'left',left_on = merged_df['item_nbr'],
                           right_on = items['item_nbr'])
    #merge with stores 
    merged_df = merged_df.merge(stores, how = 'left', left_on = merged_df['store_nbr_x'],
                           right_on = stores['store_nbr'])
    #drop non-unique columns
    _, i = np.unique(merged_df.columns, return_index = True)
    merged_df = merged_df.iloc[:,i]
    
    #drop duplicate columns with different ending 
    merged_df = merged_df.drop(['date_y','item_nbr_y','store_nbr_x','store_nbr_y','type_y',
                            'description'], axis=1)
    
    return merged_df

# Data Preprocessing

In [16]:
#merge train and test data
merged_df_train = merge_all_columns(train,transactions,oil,holiday_events,items,stores)
merged_df_test = merge_all_columns(test,transactions,oil,holiday_events,items,stores)

In [ ]:
#convert dates to ordinal values 
merged_df_train.date = merged_df_train.date.apply(lambda x: x.toordinal())
merged_df_test.date = merged_df_test.date.apply(lambda x: x.toordinal())

In [18]:
#function to create dummy variables from 'Family' column 
def create_dummies(table,column):
    old_table = table
    X_dummies = pd.get_dummies(column).iloc[:,1:]
    table = pd.concat([table,X_dummies],axis=1)
    return table

In [19]:
#Merge Train and Test Data 
merged_df_train = create_dummies(merged_df_train,merged_df_train.family)
merged_df_train = merged_df_train.drop(['family'],axis=1)
merged_df_test = create_dummies(merged_df_test,merged_df_test.family)
merged_df_test = merged_df_test.drop(['family'],axis=1)

In [77]:
#temp -- training data with no joins 
merged_df_train = train 
merged_df_test = test

In [9]:
#set train id as index 
merged_df_test = merged_df_test.set_index('id',inplace=False)
merged_df_train = merged_df_train.set_index('id',inplace=False)

In [92]:
merged_df_test.date = merged_df_test.date.apply(lambda x: x.toordinal())

## Split Training Data

In [94]:
#Split into X and y values 
y = merged_df_train['unit_sales']
X = merged_df_train.loc[:,merged_df_train.columns != 'unit_sales']

In [95]:
#temp -- drop non-date values
X = X.drop(['id','store_nbr','item_nbr','onpromotion'],axis=1)
merged_df_test = merged_df_test.drop(['id','store_nbr','item_nbr','onpromotion'],axis=1)

In [26]:
#temp -- conver date to ordinal and drop non-date values
merged_df_test.date = merged_df_test.date.apply(lambda x: x.toordinal())
merged_df_test = merged_df_test.drop(['store_nbr','item_nbr','onpromotion'],axis=1)

In [ ]:
#drop values not currently in use
X = X.drop(['city','class','item_nbr_x','store_nbr','locale_name','locale',
           'state','cluster','type_x','dcoilwtico','onpromotion','transactions','transferred']
           ,axis=1)

In [ ]:
merged_df_test = merged_df_test.drop(['city','class','item_nbr_x','store_nbr','locale_name','locale',
           'state','cluster','type_x','dcoilwtico','onpromotion','transactions','transferred']
           ,axis=1)

# Apply Algorithms

In [19]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split

In [97]:
dt = DecisionTreeRegressor(random_state = 0)
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.75,random_state=0)

In [98]:
dt.fit(X_train,y_train)

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_split=1e-07,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, presort=False, random_state=0,
           splitter='best')

In [99]:
#predict validations set 
y_pred = dt.predict(X_test)

In [23]:
X_test.columns

Index(['date'], dtype='object')

In [ ]:
merged_df_test.columns

In [ ]:
pd.DataFrame(y_pred)

In [103]:
#predict testset 
y_pred_testset = dt.predict(merged_df_test)

In [105]:
y_pred_testset_df = pd.DataFrame(y_pred_testset)

,0
0,10.318517
1,10.318517
2,10.318517
3,10.318517
4,10.318517
5,10.318517
6,10.318517
7,10.318517
8,10.318517
9,10.318517


In [50]:
y_pred_testset_df.index.name = 'id'

In [54]:
y_pred_testset_df.head()

,unit_sales
id,
0,10.318517
1,10.318517
2,10.318517
3,10.318517
4,10.318517


In [53]:
y_pred_testset_df.columns = ['unit_sales']

In [55]:
y_pred_testset_df.to_csv('date_2016_predictions.csv')

In [69]:
y_pred_testset_df.shape

(3370464, 1)

In [36]:
sample_submission.head()

,id,unit_sales
0,125497040,0
1,125497041,0
2,125497042,0
3,125497043,0
4,125497044,0


In [114]:
y_pred_testset_df

,0
0,10.318517
1,10.318517
2,10.318517
3,10.318517
4,10.318517
5,10.318517
6,10.318517
7,10.318517
8,10.318517
9,10.318517


In [125]:
y_pred_testset_df

,0
0,10.318517
1,10.318517
2,10.318517
3,10.318517
4,10.318517
5,10.318517
6,10.318517
7,10.318517
8,10.318517
9,10.318517


In [126]:
csvfile = pd.DataFrame({'id':pd.DataFrame(test.values)[0].astype(int), 
                       'unit_sales':y_pred_testset_df[0]}).set_index('id')
csvfile.to_csv('date_2016_predictions_3.csv') 

In [128]:
csvfile.shape

(3370464, 1)

In [106]:
sample_submission.head()

,id,unit_sales
0,125497040,0
1,125497041,0
2,125497042,0
3,125497043,0
4,125497044,0


In [ ]:
#Impute Missing Values -- did not work for transactions data 
from sklearn.preprocessing import Imputer 
imputer = Imputer(missing_values='NaN', strategy = 'most_frequent', axis = 0)
imputer = imputer.fit(X.iloc[:,2])
X.iloc[:,2].values = imputer.transform(X.iloc[:,2].values)

In [17]:
train['item_nbr'].value_counts()

502331     19092
314384     19082
1463810    19062
364606     19055
323013     18988
222879     18981
265559     18967
1162382    18960
414353     18892
1463814    18876
1958216    18876
1036689    18865
1463855    18850
273528     18794
1146795    18765
1958181    18759
559870     18755
564533     18754
1463860    18745
2010511    18740
847863     18722
364724     18674
1459058    18672
1464217    18671
261052     18649
885543     18629
1052563    18628
514144     18611
587069     18560
2026801    18554
           ...  
2081095       40
2081139       37
2108569       35
2105347       34
2057231       33
2068536       32
2101795       27
2103250       24
2028217       24
2111870       23
1976284       23
2112404       20
2099230       20
2090125       19
2076761       19
2081175       19
2042947       15
2088860       12
2003392       11
2089276       11
638327        10
2043198       10
2075733       10
2108843        9
2058907        8
2091318        8
2028403        6
2056549       

In [19]:
items.shape

(4100, 4)

In [13]:
train.head()

,id,date,store_nbr,item_nbr,unit_sales,onpromotion
66458908,66458908,2016-01-01,25,105574,12.0,False
66458909,66458909,2016-01-01,25,105575,9.0,False
66458910,66458910,2016-01-01,25,105857,3.0,False
66458911,66458911,2016-01-01,25,108634,3.0,False
66458912,66458912,2016-01-01,25,108701,2.0,True


In [20]:
train_groupbyid = train.groupby('item_nbr',as_index = False).sum()

In [22]:
train_groupbyid.loc[:,['item_nbr','unit_sales']].head()

,item_nbr,unit_sales
0,99197,4196.0
1,103501,38330.0
2,103520,43127.0
3,103665,47728.0
4,105574,124787.0


In [24]:
test['item_nbr'].value_counts()

317285     864
2123410    864
322095     864
264752     864
264753     864
1333307    864
1386506    864
264751     864
2048323    864
903955     864
1958166    864
1151128    864
2033992    864
1388559    864
1972587    864
371218     864
1726956    864
1583522    864
1958171    864
1583523    864
664545     864
1988975    864
1583526    864
1958175    864
1239294    864
1400846    864
268834     864
664539     864
838406     864
633735     864
          ... 
1333225    864
406965     864
853195     864
2113914    864
1417154    864
849135     864
1047695    864
1458088    864
1047696    864
849136     864
1167188    864
849139     864
1047698    864
853196     864
1047699    864
1988256    864
1047707    864
452022     864
1047702    864
849142     864
1047703    864
1464237    864
314879     864
1047705    864
1464239    864
265700     864
464263     864
1464238    864
378332     864
1157685    864
Name: item_nbr, dtype: int64